In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import shap

In [2]:
num_epochs = 10
learning_rate = 0.001
batch_size = 100
num_classes = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
    transforms.Resize(224),  
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,)),  # Normalize grayscale images
])
print(device)
model_save_path = "resnet18_fmnist.pth"

cuda


In [3]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [4]:
model = torchvision.models.resnet18(pretrained=False)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 
model.fc = torch.nn.Linear(model.fc.in_features, 10)  
 #   if isinstance(module, nn.ReLU):
  #      module.inplace = False
model.load_state_dict(torch.load('resnet18_fmnist.pth'))  # Load your trained model weights
model = model.to(device)
model.eval() 

/home/uav/anaconda3/envs/thor/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/uav/anaconda3/envs/thor/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
X_test = next(iter(test_loader))[0]
test_images = torch.tensor(X_test).to(device)
e = shap.DeepExplainer(model,test_images)

shap_values = e.shap_values(test_images, check_additivity=False)

# test_images_numpy = test_images.permute(0, 2, 3, 1).cpu().numpy() 

shap.image_plot([shap_values], test_images_numpy)

/tmp/ipykernel_131094/2615449342.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_images = torch.tensor(X_test).to(device)
2024-11-01 18:40:17.556825: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 18:40:17.562762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 18:40:17.569002: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
20

RuntimeError: Output 0 of BackwardHookFunctionBackward is a view and is being modified inplace. This view was created inside a custom Function (or because an input was returned as-is) and the autograd logic to handle view+inplace would override the custom backward associated with the custom Function, leading to incorrect gradients. This behavior is forbidden. You can fix this by cloning the output of the custom Function.

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import shap
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Helper function to disable inplace operations for all ReLU layers
def set_relu_to_non_inplace(model):
    for module in model.modules():
        if isinstance(module, nn.ReLU):
            module.inplace = False

# Load and configure ResNet-18 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 10
model_save_path = "resnet18_fmnist.pth"

model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model.to(device)
model.eval()

# Apply non-inplace ReLU setting
set_relu_to_non_inplace(model)

# Test Data Preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

# Selecting background and test samples
background = next(iter(test_loader))[0][:100].to(device)
test_images = next(iter(test_loader))[0][:5].to(device)

# Initialize SHAP DeepExplainer and compute SHAP values
explainer = shap.DeepExplainer(model, background)
shap_values = explainer.shap_values(test_images, check_additivity=False)

# Convert test images for SHAP plotting
test_images_numpy = test_images.permute(0, 2, 3, 1).cpu().numpy()

# Plot SHAP values
shap.image_plot(shap_values, test_images_numpy)



RuntimeError: Output 0 of BackwardHookFunctionBackward is a view and is being modified inplace. This view was created inside a custom Function (or because an input was returned as-is) and the autograd logic to handle view+inplace would override the custom backward associated with the custom Function, leading to incorrect gradients. This behavior is forbidden. You can fix this by cloning the output of the custom Function.